In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90.000,90.000,size=1500)
lngs = np.random.uniform(-180.000,180.000,size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
for coordinate in coordinates:
    print(coordinate[0],coordinate[1])

-75.24642718493925 21.667728953025687
49.65493162656989 -76.70093197807084
4.663422290286519 -176.96058700944735
-1.3650685420353028 91.08975862712083
10.31544482038818 13.372563848459748
18.52697228419997 37.91928390052834
66.33428810642931 -81.22029679321032
-23.391955280560012 -172.2778094158247
61.44373348423943 162.216204507398
-75.88995924206222 8.890298600439195
33.24065217718673 170.30160880205403
-74.73500239336718 165.79346773002487
9.214698490578272 -30.536589797500568
0.15670098167905167 102.06695399547363
60.24704445196201 -143.38763563237296
31.956216821377552 38.45868656585023
42.717643318284445 -30.5158825311255
67.46958551955962 119.06979255684399
86.86493412234728 64.09706647762624
59.11009703222106 164.23907892862024
-4.384754658982132 -57.15710965825134
1.1046561090324047 31.24130887051848
-72.62247159956709 65.70098882465112
41.24445735113912 -109.42779249537918
84.42935427325241 17.999647086727123
87.74019518553405 51.42955961908433
-3.865025707936482 -24.53152960

In [5]:
from citipy import citipy

In [6]:
cities = []

# Identify nearest cities for lats and longs
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    # Add each unique city to list
    if city not in cities:
        cities.append(city)

len(cities)

622